## Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

import pandas as pd
import sklearn as sk

import core.dataframe_modeler as dfmod
import helpers.dbg as dbg
import helpers.env as env
import helpers.printing as prnt
import vendors2.kibot.data.load.futures_forward_contracts as vkdlfu
import vendors2.kibot.data.load.s3_data_loader as vkdls3
import vendors2.kibot.metadata.load.kibot_metadata as vkmlki

In [ ]:
dbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", env.get_system_signature()[0])

prnt.config_notebook()

## Map contracts to start and end dates

In [ ]:
lfc_hc = vkmlki.KibotHardcodedContractLifetimeComputer(365, 7)

lfc_hc.compute_lifetime("CLJ17")

In [ ]:
lfc_ta = vkmlki.KibotTradingActivityContractLifetimeComputer()

lfc_ta.compute_lifetime("CLJ17")

In [ ]:
symbols = ["ES", "CL"]
file = "../contracts.csv"


fcl = vkmlki.FuturesContractLifetimes(file, lfc_hc)

In [ ]:
fcl.save(["CL"])

In [ ]:
cl_data = fcl.load(["CL"])

In [ ]:
cl_data["CL"].head()

## Create continuous contracts

In [ ]:
fcem = vkmlki.FuturesContractExpiryMapper(cl_data)

In [ ]:
fcem.get_nth_contract("CL", "2010-01-01", 1)

In [ ]:
srs = fcem.get_nth_contracts("CL", "2010-01-10", "2010-01-20", freq="B", n=1)

In [ ]:
srs

In [ ]:
kdl = vkdls3.S3KibotDataLoader()

In [ ]:
ffc_obj = vkdlfu.FuturesForwardContracts(kdl)

In [ ]:
ffc_obj._replace_contracts_with_data(srs)

## Combine front and back contracts

In [ ]:
cl_df = fcem.get_contracts(["CL1", "CL2", "CL3", "CL4"],
                           "2010-01-01",
                           "2015-12-31",
                           freq="B")

In [ ]:
cl_df.head()

In [ ]:
price_df = ffc_obj.replace_contracts_with_data(cl_df, "close")

In [ ]:
price_df.plot()

In [ ]:
dfm.compute_ret_0()

In [ ]:
dfm = dfmod.DataFrameModeler(df=price_df, oos_start="2013-01-01") \
          .compute_ret_0()

In [ ]:
dfm.oos_start

In [ ]:
dfm.plot_time_series()

In [ ]:
dfm.plot_pca_components()

In [ ]:
dfm.plot_explained_variance()

In [ ]:
dfm.df

In [ ]:
res = dfm.apply_residualizer(
    model_func=sk.decomposition.PCA,
    x_vars=["CL1_ret_0", "CL2_ret_0", "CL3_ret_0", "CL4_ret_0"],
    model_kwargs={
        "n_components": 1
    }
)

In [ ]:
res.df.plot()